# CS440/ECE448 Spring 2023
# MP02: Naive Bayes

The first thing you need to do is to download mp02.zip. Content is similar to MP01.

This file (`mp02_notebook.ipynb`) will walk you through the whole MP, giving you instructions and debugging tips as you go.

### Table of Contents

1. <a href="#section1">Reading the Data</a>
1. <a href="#section2">Learning a Naive Bayes Model: Maximum Likelihood</a>
1. <a href="#section3">Learning a Naive Bayes Model: Stop Words</a>
1. <a href="#section4">Learning a Naive Bayes Model: Laplace Smoothing</a>
1. <a href="#section5">Decisions Using a Naive Bayes Model</a>
1. <a href="#section6">Optimizing Hyperparameters</a>
1. <a href="#grade">Grade Your Homework</a>

<a id='section1'></a>

## Reading the data

The dataset in your template package consists of 10000 positive and 3000 negative movie reviews. It is a subset of the <a href="https://ai.stanford.edu/~amaas/data/sentiment/">Stanford Movie Review Dataset</a>, which was originally introduced by <a href="https://www.aclweb.org/anthology/P11-1015">this paper</a>. We have split this data set for you into 5000 development examples and 8000 training examples. The autograder also has a hidden set of test examples, generally similar to the development dataset. 

The data folder is structured like this:

```
  ├─ train
  │   ├─ neg
  │   │   └─ 2000 negative movie reviews (text)
  │   └─ pos
  │       └─ 6000 positive movie reviews (text)
  └─ dev
      ├─ neg
      │   └─ 1000 negative movie reviews (text)
      └─ pos
  │       └─ 4000 positive movie reviews (text)
```

In order to help you load the data, we provide you with a utility function called `reader.py`.  This has two new functions that didn't exist in mp01:

* loadTrain: load a training set
* loadDev: load a dev set

In [1]:
import reader, importlib
importlib.reload(reader)
help(reader.loadTrain)

Help on function loadTrain in module reader:

loadTrain(dirname, stemming, lower_case, use_tqdm=True)
    Loads a training dataset.
    
    Parameters:
    dirname (str): the directory containing the data
        - dirname/y should contain training examples from class y
    
    stemming (bool): if True, use NLTK's stemmer to remove suffixes
    lower_case (bool): if True, convert letters to lowercase
    use_tqdm (bool, default:True): if True, use tqdm to show status bar
    
    Output:
    train (dict of list of lists): 
        - train[y][i][k] = k'th token of i'th text of class y



This time the text files have not been lowercased for you in advance, so you probably want to lowercase them using the `lower_case` bool:

In [2]:
importlib.reload(reader)

train = reader.loadTrain('data/train', False, True)


100%|████████████████████████████████████████████████████████████████████████████| 6000/6000 [00:02<00:00, 2321.83it/s]


In [3]:
for y in train.keys():
    print("There were",len(train[y]),"texts loaded for class",y)

There were 2000 texts loaded for class neg
There were 6000 texts loaded for class pos


In [4]:
print("The first positive review is:",train['pos'][0])

The first positive review is: ['i', 'went', 'and', 'saw', 'this', 'movie', 'last', 'night', 'after', 'being', 'coaxed', 'to', 'by', 'a', 'few', 'friends', 'of', 'mine', 'i', 'll', 'admit', 'that', 'i', 'was', 'reluctant', 'to', 'see', 'it', 'because', 'from', 'what', 'i', 'knew', 'of', 'ashton', 'kutcher', 'he', 'was', 'only', 'able', 'to', 'do', 'comedy', 'i', 'was', 'wrong', 'kutcher', 'played', 'the', 'character', 'of', 'jake', 'fischer', 'very', 'well', 'and', 'kevin', 'costner', 'played', 'ben', 'randall', 'with', 'such', 'professionalism', 'the', 'sign', 'of', 'a', 'good', 'movie', 'is', 'that', 'it', 'can', 'toy', 'with', 'our', 'emotions', 'this', 'one', 'did', 'exactly', 'that', 'the', 'entire', 'theater', 'which', 'was', 'sold', 'out', 'was', 'overcome', 'by', 'laughter', 'during', 'the', 'first', 'half', 'of', 'the', 'movie', 'and', 'were', 'moved', 'to', 'tears', 'during', 'the', 'second', 'half', 'while', 'exiting', 'the', 'theater', 'i', 'not', 'only', 'saw', 'many', 'wom

<a id='section2'></a>

## Learning a Naive Bayes Model: Maximum Likelihood

In order to understand Naive Bayes, it might be useful to know the difference between bigram types and bigram tokens.

* **Bigram token:**  A bigram token consists of two consecutive word tokens from the text. For grading purpose, all bigrams are represented as word1\*\-\*\-\*\-\*word2
, i.e. \*\-\*\-\*\-\* as a separator, and not as a tuple. In the context of bigrams, the tokens are these pairs of words. To emphasize, note that our definition of bigram is two "consecutive" word tokens. Consecutive!
* **Bigram type:** Bigram types are the set of unique pairs of consecutive words that occurred in a text. The number of bigram types in the $n^{\text{th}}$ positive text can be found by first generating all bigram tokens from the text and then counting the unique bigrams.



A Naive Bayes model consists of two types of probability distributions:

* The **prior** is the distribution over classes, $P(\text{Class})$.
* The **likelihood** is the probability of a bigram token given a particular class, $P(\text{Bigram}|\text{Class})$.

The prior can be estimated from the training data. In your training data, $P(\text{Class}=\text{pos})=0.75$.  

Often, though, the testing data will have a different class distribution than the training data.  If you don't know the testing priors, then it's sometimes best to just assume a uniform distribution, i.e., $P(\text{Class}=\text{pos})=0.5$. 

The likelihood is the informative part of a Naive Bayes model: it tells you which words are used more often in negative versus positive movie reviews.

There are many ways in which you can estimate the likelihood.  The following formula is called the **maximum likelihood** estimate, because it maximizes the likelihood of the words in your training dataset:

$$P(\text{Bigram}=x|\text{Class}=y)=\frac{\text{\# tokens of bigram}~x~\text{in texts of class}~y}{\text{\# tokens of any bigram in texts of class}~y}$$



In this part of the MP, you will estimate what are called **frequency tables**.  The frequency of $x$ given $y$ is the number of times that bigram $x$ occurred in texts of class $y$.  The relevant method in `submitted.py` is the one called `create_frequency_table`:

In [5]:
import submitted, importlib
importlib.reload(submitted)
help(submitted.create_frequency_table)

Help on function create_frequency_table in module submitted:

create_frequency_table(train)
    Parameters:
    train (dict of list of lists) 
        - train[y][i][k] = k'th token of i'th text of class y
    
    Output:
    frequency (dict of Counters): 
        - frequency[y][x] = number of occurrences of bigram x in texts of class y,
          where x is in the format 'word1*-*-*-*word2'



Edit `create_frequency_table` so that it does what its docstring says it should do.  

**Hint:** your code will be shorter if you use the python data structure called a <a href="https://docs.python.org/3/library/collections.html#collections.Counter">Counter</a>.

When your code works, you should get the following results:

In [6]:
importlib.reload(submitted)
frequency = submitted.create_frequency_table(train)

print("frequency['pos'][('this', 'film')]=",frequency['pos']['this*-*-*-*film'])
print("frequency['neg'][('this', 'film')]=",frequency['neg']['this*-*-*-*film'])
print("\n")

print("frequency['pos'][('the', 'movie')]=",frequency['pos']['the*-*-*-*movie'])
print("frequency['neg'][('the', 'movie')]=",frequency['neg'][('the*-*-*-*movie')])
print("\n")


print("frequency['pos'][('of', 'the')]=",frequency['pos']['of*-*-*-*the'])
print("frequency['neg'][('of', 'the')]=",frequency['neg']['of*-*-*-*the'])
print("\n")

print("frequency['pos'][('to', 'be')]=",frequency['pos']['to*-*-*-*be'])
print("frequency['neg'][('to', 'be')]=",frequency['neg']['to*-*-*-*be'])
print("\n")

print("frequency['pos'][('and', 'the')]=",frequency['pos']['and*-*-*-*the'])
print("frequency['neg'][('and', 'the')]=",frequency['neg']['and*-*-*-*the'])
print("\n")

print("--------------------------------------\n")

print("Total # tokens in pos texts is",sum(frequency['pos'].values()))
print("Total # tokens in neg texts is",sum(frequency['neg'].values()))
print("\n")

print("Total # types in pos texts is",len(frequency['pos'].keys()))
print("Total # types in neg texts is",len(frequency['neg'].keys()))

frequency['pos'][('this', 'film')]= 2656
frequency['neg'][('this', 'film')]= 879


frequency['pos'][('the', 'movie')]= 2507
frequency['neg'][('the', 'movie')]= 1098


frequency['pos'][('of', 'the')]= 10008
frequency['neg'][('of', 'the')]= 2779


frequency['pos'][('to', 'be')]= 2500
frequency['neg'][('to', 'be')]= 1057


frequency['pos'][('and', 'the')]= 3342
frequency['neg'][('and', 'the')]= 983


--------------------------------------

Total # tokens in pos texts is 1421513
Total # tokens in neg texts is 468194


Total # types in pos texts is 475651
Total # types in neg texts is 195021


In [6]:
# gold
importlib.reload(submitted)
frequency = submitted.create_frequency_table(train)

print("frequency['pos'][('this', 'film')]=",frequency['pos']['this*-*-*-*film'])
print("frequency['neg'][('this', 'film')]=",frequency['neg']['this*-*-*-*film'])
print("\n")

print("frequency['pos'][('the', 'movie')]=",frequency['pos']['the*-*-*-*movie'])
print("frequency['neg'][('the', 'movie')]=",frequency['neg'][('the*-*-*-*movie')])
print("\n")


print("frequency['pos'][('of', 'the')]=",frequency['pos']['of*-*-*-*the'])
print("frequency['neg'][('of', 'the')]=",frequency['neg']['of*-*-*-*the'])
print("\n")

print("frequency['pos'][('to', 'be')]=",frequency['pos']['to*-*-*-*be'])
print("frequency['neg'][('to', 'be')]=",frequency['neg']['to*-*-*-*be'])
print("\n")

print("frequency['pos'][('and', 'the')]=",frequency['pos']['and*-*-*-*the'])
print("frequency['neg'][('and', 'the')]=",frequency['neg']['and*-*-*-*the'])
print("\n")

print("--------------------------------------\n")

print("Total # tokens in pos texts is",sum(frequency['pos'].values()))
print("Total # tokens in neg texts is",sum(frequency['neg'].values()))
print("\n")

print("Total # types in pos texts is",len(frequency['pos'].keys()))
print("Total # types in neg texts is",len(frequency['neg'].keys()))

frequency['pos'][('this', 'film')]= 2656
frequency['neg'][('this', 'film')]= 879


frequency['pos'][('the', 'movie')]= 2507
frequency['neg'][('the', 'movie')]= 1098


frequency['pos'][('of', 'the')]= 10008
frequency['neg'][('of', 'the')]= 2779


frequency['pos'][('to', 'be')]= 2500
frequency['neg'][('to', 'be')]= 1057


frequency['pos'][('and', 'the')]= 3342
frequency['neg'][('and', 'the')]= 983


--------------------------------------

Total # tokens in pos texts is 1421513
Total # tokens in neg texts is 468194


Total # types in pos texts is 475651
Total # types in neg texts is 195021


## Learning a Naive Bayes model: Stop words

There are many common word pairs (bigrams) that may seem to be unrelated to whether a movie review is positive or negative. Due to the nature of the training data, it's possible that some bigrams, consisting entirely of common words like "is", "of", "and", etc., are more frequent in one part of the training data than another. This can be problematic, as it means a test review might be wrongly classified as "positive" just because it contains many instances of innocuous bigrams like ("and", "the").

A "stopword list" is a list of words that should not be considered when you classify a test text. In the context of bigrams, we consider a bigram as a stopword if both of its constituent words are in the stopword list. There are many candidate stopword lists available on the internet. The stopword list that we've provided for you is based on this one: https://www.ranks.nl/stopwords.

To emphasize, we consider a bigram as a stopword if "both" of its constituent words are in the stopword list. Both!

Here is our stopword list: 

In [7]:
importlib.reload(submitted)
print(sorted(submitted.stopwords))

["'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'cannot', 'could', 'couldn', 'did', 'didn', 'do', 'does', 'doesn', 'doing', 'don', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', 'has', 'hasn', 'have', 'haven', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', 'it', 'its', 'itself', 'let', 'll', 'me', 'more', 'most', 'mustn', 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', 'shan', 'she', 'should', 'shouldn', 'so', 'some', 'such', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 

To effectively avoid counting bigrams that are considered stopwords, two steps are necessary:

1. Pretend that the frequency of bigrams, where both words are stopwords, in the training corpus is zero.
2. Ignore such bigrams if they occur in testing data.

In this part of the MP, you should set the frequencies of those bigram stopwords to zero.  Use the `del` command (see <a href="https://docs.python.org/3/library/collections.html#collections.Counter">Counters</a>), so that these bigrams don't get counted among either the bigram types or the bigram tokens.

In [8]:
importlib.reload(submitted)
help(submitted.remove_stopwords)

Help on function remove_stopwords in module submitted:

remove_stopwords(frequency)
    Parameters:
    frequency (dict of Counters): 
        - frequency[y][x] = number of occurrences of bigram x in texts of class y,
          where x is in the format 'word1*-*-*-*word2'
    stopwords (set of str):
        - Set of stopwords to be excluded
    
    Output:
    nonstop (dict of Counters): 
        - nonstop[y][x] = frequency of bigram x in texts of class y,
          but only if neither token in x is a stopword. x is in the format 'word1*-*-*-*word2'



In [9]:
importlib.reload(submitted)
nonstop = submitted.remove_stopwords(frequency)

print("frequency['pos'][('this', 'film')]=",frequency['pos']['this*-*-*-*film'])
print("frequency['pos'][('this', 'film')]=",nonstop['pos']['this*-*-*-*film'])
print("\n")

print("frequency['pos'][('the', 'movie')]=",frequency['pos']['the*-*-*-*movie'])
print("frequency['pos'][('the', 'movie')]=",nonstop['pos']['the*-*-*-*movie'])
print("\n")


print("frequency['pos'][('of', 'the')]=",frequency['pos']['of*-*-*-*the'])
print("frequency['pos'][('of', 'the')]=",nonstop['pos']['of*-*-*-*the'])
print("\n")

print("frequency['pos'][('to', 'be')]=",frequency['pos']['to*-*-*-*be'])
print("frequency['pos'][('to', 'be')]=",nonstop['pos']['to*-*-*-*be'])
print("\n")

print("frequency['pos'][('and', 'the')]=",frequency['pos']['and*-*-*-*the'])
print("frequency['pos'][('and', 'the')]=",nonstop['pos']['and*-*-*-*the'])
print("\n")

print("--------------------------------------\n")

print("Total pos frequency:",sum(frequency['pos'].values()))
print("Total pos non-stopwords",sum(nonstop['pos'].values()))
print("\n")

print("Total # types in pos texts is",len(frequency['pos'].keys()))
print("Total # non-stopwords in pos is",len(nonstop['pos'].keys()))

print("Length of the stopwords set is:",len(submitted.stopwords))

frequency['pos'][('this', 'film')]= 2656
frequency['pos'][('this', 'film')]= 2656


frequency['pos'][('the', 'movie')]= 2507
frequency['pos'][('the', 'movie')]= 2507


frequency['pos'][('of', 'the')]= 10008
frequency['pos'][('of', 'the')]= 0


frequency['pos'][('to', 'be')]= 2500
frequency['pos'][('to', 'be')]= 0


frequency['pos'][('and', 'the')]= 3342
frequency['pos'][('and', 'the')]= 0


--------------------------------------

Total pos frequency: 1421513
Total pos non-stopwords 1168682


Total # types in pos texts is 475651
Total # non-stopwords in pos is 468246
Length of the stopwords set is: 150


In [68]:
# gold
importlib.reload(submitted)
nonstop = submitted.remove_stopwords(frequency)

print("frequency['pos'][('this', 'film')]=",frequency['pos']['this*-*-*-*film'])
print("frequency['pos'][('this', 'film')]=",nonstop['pos']['this*-*-*-*film'])
print("\n")

print("frequency['pos'][('the', 'movie')]=",frequency['pos']['the*-*-*-*movie'])
print("frequency['pos'][('the', 'movie')]=",nonstop['pos']['the*-*-*-*movie'])
print("\n")


print("frequency['pos'][('of', 'the')]=",frequency['pos']['of*-*-*-*the'])
print("frequency['pos'][('of', 'the')]=",nonstop['pos']['of*-*-*-*the'])
print("\n")

print("frequency['pos'][('to', 'be')]=",frequency['pos']['to*-*-*-*be'])
print("frequency['pos'][('to', 'be')]=",nonstop['pos']['to*-*-*-*be'])
print("\n")

print("frequency['pos'][('and', 'the')]=",frequency['pos']['and*-*-*-*the'])
print("frequency['pos'][('and', 'the')]=",nonstop['pos']['and*-*-*-*the'])
print("\n")

print("--------------------------------------\n")

print("Total pos frequency:",sum(frequency['pos'].values()))
print("Total pos non-stopwords",sum(nonstop['pos'].values()))
print("\n")

print("Total # types in pos texts is",len(frequency['pos'].keys()))
print("Total # non-stopwords in pos is",len(nonstop['pos'].keys()))

print("Length of the stopwords set is:",len(submitted.stopwords))

frequency['pos'][('this', 'film')]= 2656
frequency['pos'][('this', 'film')]= 2656


frequency['pos'][('the', 'movie')]= 2507
frequency['pos'][('the', 'movie')]= 2507


frequency['pos'][('of', 'the')]= 10008
frequency['pos'][('of', 'the')]= 0


frequency['pos'][('to', 'be')]= 2500
frequency['pos'][('to', 'be')]= 0


frequency['pos'][('and', 'the')]= 3342
frequency['pos'][('and', 'the')]= 0


--------------------------------------

Total pos frequency: 1421513
Total pos non-stopwords 1168682


Total # types in pos texts is 475651
Total # non-stopwords in pos is 468246
Length of the stopwords set is: 150


## Learning a Naive Bayes model: Laplace Smoothing

The maximum likelihood formula results in some words having zero probability, just because they were not contained in your training data.  A better formula is given by Laplace smoothing, according to which

$$P(\text{Bigram}=x|\text{Class}=y)=\frac{\left(\text{\# tokens of bigram}~x~\text{in texts of class}~y\right)+k}{\left(\text{\# tokens of any bigram in texts of class}~y\right)+k\times\left(\text{\# of bigram types}+1\right)}$$

...where $k$ is a hyperparameter that is usually chosen by trying several different values, and choosing the value that gives you the best accuracy on your development dataset.  

The `+1` in the denominator is used to account for bigrams that were never seen in the training dataset for class $y$.  All such words are mapped to the type `OOV` (out of vocabulary), which has the likelihood

$$P(\text{Token}=\text{OOV}|\text{Class}=y)=\frac{k}{\left(\text{\# tokens of any bigram in texts of class}~y\right)+k\times\left(\text{\# of bigram types}+1\right)}$$

In this part of the MP, the method you'll create in `submitted.py` is called `laplace_smoothing`.

In [10]:
importlib.reload(submitted)
help(submitted.laplace_smoothing)

Help on function laplace_smoothing in module submitted:

laplace_smoothing(nonstop, smoothness)
    Parameters:
    nonstop (dict of Counters) 
        - nonstop[y][x] = frequency of bigram x in y, where x is in the format 'word1*-*-*-*word2'
          and neither word1 nor word2 is a stopword
    smoothness (float)
        - smoothness = Laplace smoothing hyperparameter
    
    Output:
    likelihood (dict of dicts) 
        - likelihood[y][x] = Laplace-smoothed likelihood of bigram x given y,
          where x is in the format 'word1*-*-*-*word2'
        - likelihood[y]['OOV'] = likelihood of an out-of-vocabulary bigram given y
    
    
    Important: 
    Be careful that your vocabulary only counts bigrams that occurred at least once
    in the training data for class y.



In [11]:
import numpy as np
a = np.array([1, 2, 3])
b = (a+1)/(3+2)
print(b)
print(type(float(b[0])))

[0.4 0.6 0.8]
<class 'float'>


In [12]:
importlib.reload(submitted)
likelihood = submitted.laplace_smoothing(frequency, 0.001)

print("likelihood['pos'][('this', 'film')]=",likelihood['pos']['this*-*-*-*film'])
print("likelihood['neg'][('this', 'film')]=",likelihood['neg']['this*-*-*-*film'])
print("\n")

print("likelihood['pos']['OOV']=",likelihood['pos']['OOV'])
print("likelihood['neg']['OOV']=",likelihood['neg']['OOV'])
print("\n")

print("(should be approx. 1): likelihood['pos'] sums to",sum(likelihood['pos'].values()))
print("(should be approx. 1): Likelihood['neg'] sums to",sum(likelihood['neg'].values()))


likelihood['pos'][('this', 'film')]= 0.0018678074513916727
likelihood['neg'][('this', 'film')]= 0.0018766473139073699


likelihood['pos']['OOV']= 7.032404925267997e-10
likelihood['neg']['OOV']= 2.134977450432218e-09


(should be approx. 1): likelihood['pos'] sums to 0.9999999999864526
(should be approx. 1): Likelihood['neg'] sums to 1.0000000000037939


In [11]:
# gold
importlib.reload(submitted)
likelihood = submitted.laplace_smoothing(frequency, 0.001)

print("likelihood['pos'][('this', 'film')]=",likelihood['pos']['this*-*-*-*film'])
print("likelihood['neg'][('this', 'film')]=",likelihood['neg']['this*-*-*-*film'])
print("\n")

print("likelihood['pos']['OOV']=",likelihood['pos']['OOV'])
print("likelihood['neg']['OOV']=",likelihood['neg']['OOV'])
print("\n")

print("(should be approx. 1): likelihood['pos'] sums to",sum(likelihood['pos'].values()))
print("(should be approx. 1): Likelihood['neg'] sums to",sum(likelihood['neg'].values()))



likelihood['pos'][('this', 'film')]= 0.0018678074513916727
likelihood['neg'][('this', 'film')]= 0.0018766473139073699


likelihood['pos']['OOV']= 7.032404925267997e-10
likelihood['neg']['OOV']= 2.134977450432218e-09


(should be approx. 1): likelihood['pos'] sums to 0.9999999999864526
(should be approx. 1): Likelihood['neg'] sums to 1.0000000000037939


## Decisions using a Naive Bayes model

Suppose you are given a text, which is just a list of word tokens, $x=[x_1,\ldots,x_n]$.  You want to decide whether this text is a positive movie review or a negative review.  According to decision theory, the probability of error is minimized by the following rule:

$$\text{Estimated Class}=\left\{\begin{array}{ll}
\text{pos}~\text{if}~P(\text{Class}=\text{pos}|\text{Text}=x) > 
P(\text{Class}=\text{neg}|\text{Text}=x)\\
\text{neg}~\text{if}~P(\text{Class}=\text{pos}|\text{Text}=x) < 
P(\text{Class}=\text{neg}|\text{Text}=x)\\
\text{undecided}~\text{if}~P(\text{Class}=\text{pos}|\text{Text}=x) = 
P(\text{Class}=\text{neg}|\text{Text}=x)\end{array}\right.$$

For a bigram model, the text $x$ is considered as a sequence of bigram tokens $[(x_1, x_2), (x_2, x_3), ... , (x_{n-1}, x_n)]$. The posterior probabilities $P(\text{Class}|\text{Text})$ can be estimated using the Naive Bayes model:

$$P(\text{Class}=y|\text{Text}=x)=\frac{P(\text{Class}=y)}{P(\text{Text}=x)}\prod_{i\not\in\text{stopword bigrams},i=1}^{n-1}P(\text{Token}=x_i|\text{Class}=y)$$


#### Implementation Details

Notice some details:
1. The term $P(\text{Text}=x)$ doesn't depend on $y$.  If you're trying to figure out which is bigger, $P(\text{pos}|x)$ or $P(\text{neg}|x)$, then you don't need to calculate it.
1. Multiplying together $n$ probabilities will result in a number that your computer might round down to 0.  In order to prevent that, take the logarithm of both sides of the equation above.
1. If $x_i$ is a stopword bigram, don't calculate its likelihood.  If it isn't a stopword bigram, but it doesn't have an entry in `likelihood[y]`, then you should use `likelihood[y]["OOV"]`.

#### Implementation

For this part of the MP, finish the method called `submitted.naive_bayes`:

In [13]:
importlib.reload(submitted)
help(submitted.naive_bayes)

Help on function naive_bayes in module submitted:

naive_bayes(texts, likelihood, prior)
    Parameters:
    texts (list of lists) -
        - texts[i][k] = k'th token of i'th text
    likelihood (dict of dicts) 
        - likelihood[y][x] = Laplace-smoothed likelihood of bigram x given y,
          where x is in the format 'word1*-*-*-*word2'
    prior (float)
        - prior = the prior probability of the class called "pos"
    
    Output:
    hypotheses (list)
        - hypotheses[i] = class label for the i'th text



Use `reader.loadDev` to load the dev set, then try classifying it with, say, a prior of 0.5:

In [14]:
importlib.reload(reader)
texts, labels = reader.loadDev('data/dev', False, True, True)

for y in ['neg','pos']:
    print("There are",labels.count(y),'examples of class',y)

100%|████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:01<00:00, 2543.10it/s]

There are 1000 examples of class neg
There are 4000 examples of class pos


In [17]:
import numpy as np
a = float(np.log(5))
print(type(a))

<class 'float'>


In [21]:
importlib.reload(submitted)
hypotheses = submitted.naive_bayes(texts, likelihood, 0.5)

for y in ['neg','pos', 'undecided']:
    print("There are",hypotheses.count(y),'examples that were labeled with class',y)

finish 0-th text, neg
finish 1-th text, pos
finish 2-th text, pos
finish 3-th text, pos
finish 4-th text, pos
finish 5-th text, pos
finish 6-th text, neg
finish 7-th text, neg
finish 8-th text, neg
finish 9-th text, neg
finish 10-th text, neg
finish 11-th text, pos
finish 12-th text, neg
finish 13-th text, neg
finish 14-th text, neg
finish 15-th text, pos
finish 16-th text, neg
finish 17-th text, pos
finish 18-th text, neg
finish 19-th text, pos
finish 20-th text, neg
finish 21-th text, neg
finish 22-th text, neg
finish 23-th text, neg
finish 24-th text, neg
finish 25-th text, neg
finish 26-th text, neg
finish 27-th text, neg
finish 28-th text, pos
finish 29-th text, pos
finish 30-th text, neg
finish 31-th text, neg
finish 32-th text, pos
finish 33-th text, pos
finish 34-th text, pos
finish 35-th text, neg
finish 36-th text, neg
finish 37-th text, neg
finish 38-th text, neg
finish 39-th text, pos
finish 40-th text, neg
finish 41-th text, pos
finish 42-th text, neg
finish 43-th text, ne

finish 875-th text, neg
finish 876-th text, neg
finish 877-th text, pos
finish 878-th text, neg
finish 879-th text, neg
finish 880-th text, pos
finish 881-th text, neg
finish 882-th text, pos
finish 883-th text, pos
finish 884-th text, neg
finish 885-th text, neg
finish 886-th text, pos
finish 887-th text, neg
finish 888-th text, neg
finish 889-th text, pos
finish 890-th text, neg
finish 891-th text, neg
finish 892-th text, neg
finish 893-th text, neg
finish 894-th text, pos
finish 895-th text, pos
finish 896-th text, pos
finish 897-th text, pos
finish 898-th text, pos
finish 899-th text, neg
finish 900-th text, neg
finish 901-th text, pos
finish 902-th text, pos
finish 903-th text, pos
finish 904-th text, neg
finish 905-th text, pos
finish 906-th text, neg
finish 907-th text, neg
finish 908-th text, neg
finish 909-th text, neg
finish 910-th text, pos
finish 911-th text, neg
finish 912-th text, neg
finish 913-th text, neg
finish 914-th text, pos
finish 915-th text, pos
finish 916-th te

finish 1601-th text, pos
finish 1602-th text, pos
finish 1603-th text, pos
finish 1604-th text, pos
finish 1605-th text, neg
finish 1606-th text, neg
finish 1607-th text, pos
finish 1608-th text, pos
finish 1609-th text, pos
finish 1610-th text, pos
finish 1611-th text, pos
finish 1612-th text, pos
finish 1613-th text, pos
finish 1614-th text, pos
finish 1615-th text, pos
finish 1616-th text, pos
finish 1617-th text, pos
finish 1618-th text, pos
finish 1619-th text, pos
finish 1620-th text, pos
finish 1621-th text, pos
finish 1622-th text, pos
finish 1623-th text, pos
finish 1624-th text, pos
finish 1625-th text, pos
finish 1626-th text, pos
finish 1627-th text, pos
finish 1628-th text, pos
finish 1629-th text, pos
finish 1630-th text, pos
finish 1631-th text, pos
finish 1632-th text, pos
finish 1633-th text, pos
finish 1634-th text, pos
finish 1635-th text, pos
finish 1636-th text, pos
finish 1637-th text, pos
finish 1638-th text, pos
finish 1639-th text, neg
finish 1640-th text, pos


finish 2434-th text, pos
finish 2435-th text, pos
finish 2436-th text, pos
finish 2437-th text, pos
finish 2438-th text, pos
finish 2439-th text, pos
finish 2440-th text, pos
finish 2441-th text, pos
finish 2442-th text, pos
finish 2443-th text, pos
finish 2444-th text, neg
finish 2445-th text, pos
finish 2446-th text, pos
finish 2447-th text, pos
finish 2448-th text, pos
finish 2449-th text, pos
finish 2450-th text, pos
finish 2451-th text, pos
finish 2452-th text, pos
finish 2453-th text, pos
finish 2454-th text, pos
finish 2455-th text, pos
finish 2456-th text, pos
finish 2457-th text, pos
finish 2458-th text, pos
finish 2459-th text, pos
finish 2460-th text, pos
finish 2461-th text, pos
finish 2462-th text, pos
finish 2463-th text, pos
finish 2464-th text, pos
finish 2465-th text, pos
finish 2466-th text, pos
finish 2467-th text, pos
finish 2468-th text, pos
finish 2469-th text, pos
finish 2470-th text, pos
finish 2471-th text, pos
finish 2472-th text, pos
finish 2473-th text, pos


finish 3327-th text, pos
finish 3328-th text, pos
finish 3329-th text, pos
finish 3330-th text, pos
finish 3331-th text, pos
finish 3332-th text, pos
finish 3333-th text, pos
finish 3334-th text, pos
finish 3335-th text, pos
finish 3336-th text, pos
finish 3337-th text, pos
finish 3338-th text, pos
finish 3339-th text, pos
finish 3340-th text, pos
finish 3341-th text, pos
finish 3342-th text, neg
finish 3343-th text, pos
finish 3344-th text, pos
finish 3345-th text, pos
finish 3346-th text, pos
finish 3347-th text, pos
finish 3348-th text, pos
finish 3349-th text, pos
finish 3350-th text, pos
finish 3351-th text, neg
finish 3352-th text, pos
finish 3353-th text, pos
finish 3354-th text, pos
finish 3355-th text, pos
finish 3356-th text, pos
finish 3357-th text, pos
finish 3358-th text, pos
finish 3359-th text, pos
finish 3360-th text, pos
finish 3361-th text, pos
finish 3362-th text, pos
finish 3363-th text, pos
finish 3364-th text, pos
finish 3365-th text, pos
finish 3366-th text, pos


finish 3920-th text, pos
finish 3921-th text, pos
finish 3922-th text, pos
finish 3923-th text, pos
finish 3924-th text, pos
finish 3925-th text, pos
finish 3926-th text, pos
finish 3927-th text, pos
finish 3928-th text, pos
finish 3929-th text, pos
finish 3930-th text, pos
finish 3931-th text, pos
finish 3932-th text, pos
finish 3933-th text, pos
finish 3934-th text, pos
finish 3935-th text, pos
finish 3936-th text, pos
finish 3937-th text, neg
finish 3938-th text, pos
finish 3939-th text, pos
finish 3940-th text, pos
finish 3941-th text, pos
finish 3942-th text, pos
finish 3943-th text, pos
finish 3944-th text, pos
finish 3945-th text, pos
finish 3946-th text, pos
finish 3947-th text, neg
finish 3948-th text, pos
finish 3949-th text, pos
finish 3950-th text, pos
finish 3951-th text, pos
finish 3952-th text, pos
finish 3953-th text, pos
finish 3954-th text, pos
finish 3955-th text, pos
finish 3956-th text, pos
finish 3957-th text, pos
finish 3958-th text, pos
finish 3959-th text, pos


finish 4590-th text, pos
finish 4591-th text, pos
finish 4592-th text, pos
finish 4593-th text, pos
finish 4594-th text, pos
finish 4595-th text, pos
finish 4596-th text, pos
finish 4597-th text, pos
finish 4598-th text, pos
finish 4599-th text, pos
finish 4600-th text, pos
finish 4601-th text, pos
finish 4602-th text, neg
finish 4603-th text, pos
finish 4604-th text, pos
finish 4605-th text, pos
finish 4606-th text, pos
finish 4607-th text, pos
finish 4608-th text, pos
finish 4609-th text, pos
finish 4610-th text, neg
finish 4611-th text, pos
finish 4612-th text, pos
finish 4613-th text, pos
finish 4614-th text, pos
finish 4615-th text, pos
finish 4616-th text, pos
finish 4617-th text, pos
finish 4618-th text, pos
finish 4619-th text, pos
finish 4620-th text, pos
finish 4621-th text, neg
finish 4622-th text, pos
finish 4623-th text, pos
finish 4624-th text, pos
finish 4625-th text, pos
finish 4626-th text, pos
finish 4627-th text, pos
finish 4628-th text, pos
finish 4629-th text, pos


In [22]:
print(len(hypotheses)) 
print(len(labels))

5000
5000


In [23]:
print("The accuracy of the classifier on the dev set is:")

count_correct = 0
for (y,yhat) in zip(labels, hypotheses):
    if y==yhat:
        count_correct += 1
        
print(count_correct / len(labels))

The accuracy of the classifier on the dev set is:
0.884


In [16]:
# gold
print("The accuracy of the classifier on the dev set is:")

count_correct = 0
for (y,yhat) in zip(labels, hypotheses):
    if y==yhat:
        count_correct += 1
        
print(count_correct / len(labels))

The accuracy of the classifier on the dev set is:
0.884


## Optimizing Hyperparameters

The performance of the model is heavily influenced by two parameters that can't be measured from the training data:

1. The prior, $P(\text{Class}=\text{pos})$.  The training and testing data might have different priors, so estimating this from the training data is suboptimal.
1. The Laplace smoothing parameter, $k$.

Since these two parameters can't be (correctly) estimated from the training data, they are called **hyperparameters**.  Hyperparameters are usually determined based on your knowledge about the problem, or by running a lot of experiments to see which values give the best result on the development test data.

The function you'll write in this part of the MP is called `optimize_hyperparameters`.

In [24]:
importlib.reload(submitted)
help(submitted.optimize_hyperparameters)

Help on function optimize_hyperparameters in module submitted:

optimize_hyperparameters(texts, labels, nonstop, priors, smoothnesses)
    Parameters:
    texts (list of lists) - dev set texts
        - texts[i][k] = k'th token of i'th text
    labels (list) - dev set labels
        - labels[i] = class label of i'th text
    nonstop (dict of Counters) 
        - nonstop[y][x] = frequency of word x in class y, x not stopword
    priors (list)
        - a list of different possible values of the prior
    smoothnesses (list)
        - a list of different possible values of the smoothness
    
    Output:
    accuracies (numpy array, shape = len(priors) x len(smoothnesses))
        - accuracies[m,n] = dev set accuracy achieved using the
          m'th candidate prior and the n'th candidate smoothness



Let's use this function to test some different candidate values for the prior and the smoothness.  The values we test are a little arbitrary, but let's try the following:

In [25]:
importlib.reload(submitted)
import numpy as np

priors = [0.5,0.65,0.75]
smoothnesses = [0.0001,0.001,0.01]
accuracies = submitted.optimize_hyperparameters(texts,labels,nonstop,priors,smoothnesses)

(m,n) = np.unravel_index(np.argmax(accuracies), accuracies.shape)
print("The best accuracy achieved was",accuracies[m,n])
print("It was achieved for a prior of", priors[m], "and a smoothness of", smoothnesses[n])

finish 0-th text, neg
finish 1-th text, pos
finish 2-th text, pos
finish 3-th text, pos
finish 4-th text, pos
finish 5-th text, pos
finish 6-th text, neg
finish 7-th text, neg
finish 8-th text, neg
finish 9-th text, neg
finish 10-th text, neg
finish 11-th text, pos
finish 12-th text, neg
finish 13-th text, pos
finish 14-th text, neg
finish 15-th text, pos
finish 16-th text, neg
finish 17-th text, pos
finish 18-th text, pos
finish 19-th text, pos
finish 20-th text, neg
finish 21-th text, neg
finish 22-th text, neg
finish 23-th text, neg
finish 24-th text, neg
finish 25-th text, pos
finish 26-th text, neg
finish 27-th text, neg
finish 28-th text, pos
finish 29-th text, pos
finish 30-th text, neg
finish 31-th text, pos
finish 32-th text, pos
finish 33-th text, pos
finish 34-th text, pos
finish 35-th text, neg
finish 36-th text, pos
finish 37-th text, neg
finish 38-th text, pos
finish 39-th text, pos
finish 40-th text, neg
finish 41-th text, pos
finish 42-th text, neg
finish 43-th text, ne

finish 808-th text, neg
finish 809-th text, pos
finish 810-th text, pos
finish 811-th text, neg
finish 812-th text, pos
finish 813-th text, pos
finish 814-th text, neg
finish 815-th text, pos
finish 816-th text, pos
finish 817-th text, pos
finish 818-th text, pos
finish 819-th text, neg
finish 820-th text, neg
finish 821-th text, neg
finish 822-th text, neg
finish 823-th text, neg
finish 824-th text, pos
finish 825-th text, neg
finish 826-th text, neg
finish 827-th text, pos
finish 828-th text, pos
finish 829-th text, neg
finish 830-th text, neg
finish 831-th text, neg
finish 832-th text, neg
finish 833-th text, neg
finish 834-th text, pos
finish 835-th text, pos
finish 836-th text, neg
finish 837-th text, neg
finish 838-th text, neg
finish 839-th text, pos
finish 840-th text, pos
finish 841-th text, pos
finish 842-th text, neg
finish 843-th text, pos
finish 844-th text, neg
finish 845-th text, pos
finish 846-th text, neg
finish 847-th text, pos
finish 848-th text, neg
finish 849-th te

finish 1718-th text, pos
finish 1719-th text, pos
finish 1720-th text, pos
finish 1721-th text, pos
finish 1722-th text, pos
finish 1723-th text, neg
finish 1724-th text, pos
finish 1725-th text, pos
finish 1726-th text, pos
finish 1727-th text, pos
finish 1728-th text, pos
finish 1729-th text, pos
finish 1730-th text, pos
finish 1731-th text, pos
finish 1732-th text, pos
finish 1733-th text, pos
finish 1734-th text, neg
finish 1735-th text, pos
finish 1736-th text, pos
finish 1737-th text, pos
finish 1738-th text, pos
finish 1739-th text, pos
finish 1740-th text, pos
finish 1741-th text, pos
finish 1742-th text, pos
finish 1743-th text, pos
finish 1744-th text, pos
finish 1745-th text, pos
finish 1746-th text, pos
finish 1747-th text, pos
finish 1748-th text, pos
finish 1749-th text, pos
finish 1750-th text, pos
finish 1751-th text, pos
finish 1752-th text, pos
finish 1753-th text, pos
finish 1754-th text, pos
finish 1755-th text, pos
finish 1756-th text, pos
finish 1757-th text, pos


finish 2434-th text, pos
finish 2435-th text, pos
finish 2436-th text, pos
finish 2437-th text, pos
finish 2438-th text, pos
finish 2439-th text, pos
finish 2440-th text, pos
finish 2441-th text, pos
finish 2442-th text, pos
finish 2443-th text, pos
finish 2444-th text, neg
finish 2445-th text, pos
finish 2446-th text, pos
finish 2447-th text, pos
finish 2448-th text, pos
finish 2449-th text, pos
finish 2450-th text, pos
finish 2451-th text, pos
finish 2452-th text, pos
finish 2453-th text, pos
finish 2454-th text, pos
finish 2455-th text, pos
finish 2456-th text, pos
finish 2457-th text, pos
finish 2458-th text, pos
finish 2459-th text, pos
finish 2460-th text, pos
finish 2461-th text, pos
finish 2462-th text, pos
finish 2463-th text, pos
finish 2464-th text, pos
finish 2465-th text, pos
finish 2466-th text, pos
finish 2467-th text, pos
finish 2468-th text, pos
finish 2469-th text, pos
finish 2470-th text, pos
finish 2471-th text, pos
finish 2472-th text, pos
finish 2473-th text, pos


finish 3201-th text, pos
finish 3202-th text, pos
finish 3203-th text, pos
finish 3204-th text, pos
finish 3205-th text, pos
finish 3206-th text, pos
finish 3207-th text, pos
finish 3208-th text, pos
finish 3209-th text, pos
finish 3210-th text, pos
finish 3211-th text, pos
finish 3212-th text, pos
finish 3213-th text, pos
finish 3214-th text, pos
finish 3215-th text, pos
finish 3216-th text, pos
finish 3217-th text, pos
finish 3218-th text, pos
finish 3219-th text, pos
finish 3220-th text, pos
finish 3221-th text, pos
finish 3222-th text, pos
finish 3223-th text, pos
finish 3224-th text, pos
finish 3225-th text, pos
finish 3226-th text, pos
finish 3227-th text, pos
finish 3228-th text, pos
finish 3229-th text, pos
finish 3230-th text, pos
finish 3231-th text, pos
finish 3232-th text, pos
finish 3233-th text, pos
finish 3234-th text, pos
finish 3235-th text, pos
finish 3236-th text, pos
finish 3237-th text, pos
finish 3238-th text, pos
finish 3239-th text, pos
finish 3240-th text, pos


...

finish 2138-th text, pos
finish 2139-th text, neg
finish 2140-th text, pos
finish 2141-th text, pos
finish 2142-th text, neg
finish 2143-th text, pos
finish 2144-th text, pos
finish 2145-th text, pos
finish 2146-th text, pos
finish 2147-th text, neg
finish 2148-th text, pos
finish 2149-th text, pos
finish 2150-th text, pos
finish 2151-th text, pos
finish 2152-th text, pos
finish 2153-th text, neg
finish 2154-th text, pos
finish 2155-th text, neg
finish 2156-th text, pos
finish 2157-th text, pos
finish 2158-th text, pos
finish 2159-th text, neg
finish 2160-th text, pos
finish 2161-th text, pos
finish 2162-th text, pos
finish 2163-th text, pos
finish 2164-th text, pos
finish 2165-th text, pos
finish 2166-th text, neg
finish 2167-th text, pos
finish 2168-th text, pos
finish 2169-th text, pos
finish 2170-th text, neg
finish 2171-th text, pos
finish 2172-th text, pos
finish 2173-th text, pos
finish 2174-th text, pos
finish 2175-th text, pos
finish 2176-th text, neg
finish 2177-th text, pos


finish 2557-th text, pos
finish 2558-th text, pos
finish 2559-th text, pos
finish 2560-th text, pos
finish 2561-th text, neg
finish 2562-th text, pos
finish 2563-th text, pos
finish 2564-th text, pos
finish 2565-th text, pos
finish 2566-th text, pos
finish 2567-th text, pos
finish 2568-th text, pos
finish 2569-th text, pos
finish 2570-th text, pos
finish 2571-th text, pos
finish 2572-th text, pos
finish 2573-th text, pos
finish 2574-th text, pos
finish 2575-th text, pos
finish 2576-th text, pos
finish 2577-th text, pos
finish 2578-th text, pos
finish 2579-th text, neg
finish 2580-th text, neg
finish 2581-th text, pos
finish 2582-th text, neg
finish 2583-th text, pos
finish 2584-th text, pos
finish 2585-th text, pos
finish 2586-th text, pos
finish 2587-th text, neg
finish 2588-th text, pos
finish 2589-th text, pos
finish 2590-th text, pos
finish 2591-th text, pos
finish 2592-th text, pos
finish 2593-th text, pos
finish 2594-th text, pos
finish 2595-th text, pos
finish 2596-th text, pos


finish 3046-th text, pos
finish 3047-th text, pos
finish 3048-th text, neg
finish 3049-th text, pos
finish 3050-th text, pos
finish 3051-th text, pos
finish 3052-th text, neg
finish 3053-th text, pos
finish 3054-th text, pos
finish 3055-th text, pos
finish 3056-th text, pos
finish 3057-th text, neg
finish 3058-th text, pos
finish 3059-th text, neg
finish 3060-th text, pos
finish 3061-th text, pos
finish 3062-th text, pos
finish 3063-th text, pos
finish 3064-th text, pos
finish 3065-th text, pos
finish 3066-th text, pos
finish 3067-th text, pos
finish 3068-th text, pos
finish 3069-th text, pos
finish 3070-th text, pos
finish 3071-th text, pos
finish 3072-th text, neg
finish 3073-th text, pos
finish 3074-th text, pos
finish 3075-th text, pos
finish 3076-th text, pos
finish 3077-th text, pos
finish 3078-th text, pos
finish 3079-th text, pos
finish 3080-th text, pos
finish 3081-th text, pos
finish 3082-th text, pos
finish 3083-th text, pos
finish 3084-th text, pos
finish 3085-th text, pos


finish 3704-th text, pos
finish 3705-th text, pos
finish 3706-th text, pos
finish 3707-th text, pos
finish 3708-th text, neg
finish 3709-th text, pos
finish 3710-th text, pos
finish 3711-th text, pos
finish 3712-th text, neg
finish 3713-th text, neg
finish 3714-th text, pos
finish 3715-th text, pos
finish 3716-th text, pos
finish 3717-th text, pos
finish 3718-th text, pos
finish 3719-th text, pos
finish 3720-th text, pos
finish 3721-th text, pos
finish 3722-th text, neg
finish 3723-th text, pos
finish 3724-th text, pos
finish 3725-th text, pos
finish 3726-th text, pos
finish 3727-th text, pos
finish 3728-th text, pos
finish 3729-th text, pos
finish 3730-th text, pos
finish 3731-th text, pos
finish 3732-th text, pos
finish 3733-th text, pos
finish 3734-th text, pos
finish 3735-th text, pos
finish 3736-th text, pos
finish 3737-th text, pos
finish 3738-th text, pos
finish 3739-th text, pos
finish 3740-th text, neg
finish 3741-th text, neg
finish 3742-th text, pos
finish 3743-th text, pos


finish 4452-th text, pos
finish 4453-th text, pos
finish 4454-th text, pos
finish 4455-th text, neg
finish 4456-th text, pos
finish 4457-th text, neg
finish 4458-th text, pos
finish 4459-th text, pos
finish 4460-th text, pos
finish 4461-th text, pos
finish 4462-th text, pos
finish 4463-th text, pos
finish 4464-th text, pos
finish 4465-th text, pos
finish 4466-th text, pos
finish 4467-th text, pos
finish 4468-th text, pos
finish 4469-th text, pos
finish 4470-th text, pos
finish 4471-th text, pos
finish 4472-th text, neg
finish 4473-th text, pos
finish 4474-th text, pos
finish 4475-th text, pos
finish 4476-th text, pos
finish 4477-th text, pos
finish 4478-th text, neg
finish 4479-th text, pos
finish 4480-th text, pos
finish 4481-th text, neg
finish 4482-th text, pos
finish 4483-th text, neg
finish 4484-th text, pos
finish 4485-th text, neg
finish 4486-th text, pos
finish 4487-th text, pos
finish 4488-th text, pos
finish 4489-th text, pos
finish 4490-th text, pos
finish 4491-th text, pos


## Grade your homework

If you've reached this point, and all of the above sections work, then you're ready to try grading your homework!  Before you submit it to Gradescope, try grading it on your own machine.  This will run some visible test cases (which you can read in `tests/test_visible.py`), and compare the results to the solutions (which you can read in `solution.json`).

The exclamation point (!) tells python to run the following as a shell command.  Obviously you don't need to run the code this way -- this usage is here just to remind you that you can also, if you wish, run this command in a terminal window.

In [26]:
!python grade.py

.....
----------------------------------------------------------------------
Ran 5 tests in 18.172s

OK


If you got any 'E' marks, it means that your code generated some runtime errors, and you need to debug those.

If you got any 'F' marks, it means that your code ran without errors, but that it generated results that are different from the solutions in `solutions.json`.  Try debugging those differences.

If neither of those things happened, and your result was a series of dots, then your code works perfectly.  

If you're not sure, you can try running grade.py with the -j option.  This will produce a JSON results file, in which the best score you can get is 50.

Now you should try uploading `submitted.py` to <a href="https://www.gradescope.com/courses/486387">Gradescope</a>.  

Gradescope will run the same visible tests that you just ran on your own machine, plus some additional hidden tests.  It's possible that your code passes all the visible tests, but fails the hidden tests.  If that happens, then it probably means that you hard-coded a number into your function definition, instead of using the input parameter that you were supposed to use.  Debug by running your function with a variety of different input parameters, and see if you can get it to respond correctly in all cases.

Once your code works perfectly on Gradescope, with no errors, then you are done with the MP.  Congratulations!